# SBP Reader - Locosys SiRF Binary (Packed)

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys
import time

from datetime import datetime, timezone

import numpy as np

import unittest

from sirf_reader import SirfReader

## Main Class

In [2]:
class SbpReader(SirfReader):
    '''SBP file - Locosys SiRF Binary (Packed)'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def readHeader(self):
        '''Load Locosysy header into memory'''

        # Handle SBP files without a header. This is rare but it has been known - e.g. manually created SBP files
        if self.buffer[2] == 0xa0 and self.buffer[3] == 0xa2:

            # True length = start token + internal length + payload + checksum + end token = internal length + 8
            trueLength = self.readUnsigned16LE()

            # Read the Locosysy header
            super().readHeader()


    def readData(self):
        '''Read data into memory'''

        fileSize = len(self.buffer)

        if fileSize % 32 != 0:
            # A small handful of SBP files from the GT-11 ended with a single byte 0x0a
            if fileSize % 32 == 1 and self.buffer[fileSize - 1] == 0x0a:
                fileSize -= 1
            else:
                raise ValueError('File is not a multiple of 32 bytes')

        # Ignore 0xff padding at the end of some GT-11 files - only need to check HDOP and SV count
        while self.buffer[fileSize - 32] == 0xff and self.buffer[fileSize - 31] == 0xff:
            fileSize -= 32

        self.numRecords = (fileSize - 64) // 32

        dtype = self.getDatatype()

        self.rawData = np.frombuffer(self.buffer[64:], dtype=dtype, count=self.numRecords)
        
        self.consumeRawData()


    def getDatatype(self):
        '''Get the datatype of the SBP messages'''

        # Standard 32-byte SBN message (GT-31 onwards). Last 2 bytes were originally for a flag and unused (GT-11)
        dtype = [
            ('hdop', 'u1'),
            ('sv_count', 'u1'),
            ('utc_millisecs', '<u2'),
            ('utc_packed_datetime', '<u4'),
            ('sv_ids', '<u4'),
            ('latitude', '<i4'),
            ('longitude', '<i4'),
            ('altitude_msl', '<i4'),
            ('sog', '<u2'),
            ('cog', '<u2'),
            ('climb_rate', '<i2'),
            ('sdop', 'u1'),
            ('vsdop', 'u1')
        ]

        return np.dtype(dtype)


    def consumeRawData(self):
        '''Consume data - convert to standard data structure'''

        # Convert packed date/time to regular timestamp (seconds)
        vfunc = np.vectorize(decodePackedDateTime)
        self.data['timestamp'] = vfunc(self.rawData['utc_packed_datetime'], self.rawData['utc_millisecs'])

        # Consume universal SiRF binary fields
        super().consumeRawData()

        # Convert SDOP from cm/s to m/s - original SBP format without SDOP had "bitflags" (min = 0 and max = 1)
        if self.rawData['sdop'].min() > 0 or self.rawData['sdop'].max() > 1:
            self.data['sdop'] = self.rawData['sdop'] / 100

        # Convert VSDOP from cm/s to m/s - original SBP format without VSDOP had "reserved" (min = 0 and max = 0)
        if self.rawData['vsdop'].min() > 0 or self.rawData['vsdop'].max() > 0:
            self.data['vsdop'] = self.rawData['vsdop'] / 100


def decodePackedDateTime(packedDateTime, millisecs):
    '''Decode packed date/time'''

    second = packedDateTime & 0x3f
    packedDateTime >>= 6

    minute = packedDateTime & 0x3f
    packedDateTime >>= 6

    hour = packedDateTime & 0x1f
    packedDateTime >>= 5

    day = packedDateTime & 0x1f
    packedDateTime >>= 5

    month = packedDateTime % 12
    year = packedDateTime // 12 + 2000
    
    if month == 0:
        month = 12
        year -= 1

    # Out of 800 files in SBP format, I have only encountered one reading where millisecs // 1000 != second

    return datetime(year, month, day, hour, minute, millisecs // 1000, millisecs % 1000, tzinfo=timezone.utc).timestamp()

## Unit Tests

In [3]:
class TestDates(unittest.TestCase):
    '''Class to test dates are correctly decoded'''

    def testDec2007(self):
        '''Test that 27 Dec 2007 works as expected'''

        actual = decodePackedDateTime(0x1836da2c, 44000)

        expected = datetime(2007, 12, 27, 13, 40, 44, tzinfo=timezone.utc).timestamp()
        
        self.assertEqual(actual, expected)


    def testDec2011(self):
        '''Test that 3 Dec 2011 works as expected'''

        actual = decodePackedDateTime(0x2406cb2f, 47000)

        expected = datetime(2011, 12, 3, 12, 44, 47, tzinfo=timezone.utc).timestamp()
        
        self.assertEqual(actual, expected)


    def testApr2020(self):
        '''Test that 10 Apr 2022 works as expected'''

        actual = decodePackedDateTime(0x4314cb45, 5000)

        expected = datetime(2022, 4, 10, 12, 45, 5, tzinfo=timezone.utc).timestamp()
        
        self.assertEqual(actual, expected)

In [4]:
class TestHeader(unittest.TestCase):
    '''Class to test header was correctly loaded'''

    def testUsername(self):
        '''Test the username is as expected'''

        self.assertEqual(sbpReader.header['username'], 'GEORG30MICHA')


    def testSerial(self):
        '''Test the serial is as expected'''

        self.assertEqual(sbpReader.header['serial'], 932000175)


    def testFrequency(self):
        '''Test the frequency is as expected'''

        self.assertEqual(sbpReader.header['frequency'], 1)


    def testFirmware(self):
        '''Test the firmware is as expected'''

        self.assertEqual(sbpReader.header['firmware'], 'V1.4(B0803T)')

In [5]:
class TestData(unittest.TestCase):
    '''Class to test data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(sbpReader.numRecords, 4162)

        for fieldName in sbpReader.data:
            self.assertEqual(sbpReader.data[fieldName].size, 4162)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(sbpReader.data['hdop'].min(), 0.8)
        self.assertEqual(sbpReader.data['hdop'].max(), 2.0)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(sbpReader.data['sat'].min(), 0)
        self.assertEqual(sbpReader.data['sat'].max(), 10)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(sbpReader.data['timestamp'].min(), 1649672161)
        self.assertEqual(sbpReader.data['timestamp'].max(), 1649678793)


    def testSvIds(self):
        '''Test the satellite IDs are as expected'''

        self.assertEqual(sbpReader.data['sv_ids'].min(), 0)
        self.assertEqual(sbpReader.data['sv_ids'].max(), 3542222850)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(sbpReader.data['lat'].min(), 50.5710156)
        self.assertEqual(sbpReader.data['lat'].max(), 50.583341)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(sbpReader.data['lon'].min(), -2.4620455)
        self.assertEqual(sbpReader.data['lon'].max(), -2.4563038)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(sbpReader.data['ele'].min(), -3.06)
        self.assertEqual(sbpReader.data['ele'].max(), 11.93)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(sbpReader.data['sog'].min(), 0.01)
        self.assertEqual(sbpReader.data['sog'].max(), 16.83)


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(sbpReader.data['cog'].min(), 0.01)
        self.assertEqual(sbpReader.data['cog'].max(), 359.92)


    def testRoc(self):
        '''Test the rate of climb is as expected'''

        self.assertEqual(sbpReader.data['roc'].min(), -0.56)
        self.assertEqual(sbpReader.data['roc'].max(), 0.35)


    def testSdop(self):
        '''Test the speed dilution of precision is as expected'''

        self.assertEqual(sbpReader.data['sdop'].min(), 0.09)
        self.assertEqual(sbpReader.data['sdop'].max(), 1.65)


    def testVsdop(self):
        '''Test the vertical speed dilution of precision is as expected'''

        self.assertEqual(sbpReader.data['vsdop'].min(), 0.1)
        self.assertEqual(sbpReader.data['vsdop'].max(), 0.64)

In [6]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220512_094254_DLG.SBP')
    sbpReader = SbpReader(filename)

    pc1 = time.perf_counter()
    sbpReader.load()
    pc2 = time.perf_counter()

    print("\nSBP file loaded in in %0.2f seconds" % (pc2 - pc1))


SBP file loaded in in 0.01 seconds


In [7]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

....................
----------------------------------------------------------------------
Ran 20 tests in 0.009s

OK
